In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the parameters and settings for your CGAN
num_epochs = 1000
batch_size = 32
num_columns = 10
num_rows = 437
latent_dim = 100
# labels1 = np.mean(data, axis=1).reshape(-1, 1)

data = pd.read_csv('data.csv')
data = data.values.astype('float32')

labels1 = np.mean(data, axis=1).reshape(-1, 1)

In [26]:
def define_generator():
    model = keras.Sequential()
    model.add(layers.Dense(30, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(num_columns, activation='sigmoid'))
    return model
def define_discriminator():
    model = keras.Sequential()
    model.add(layers.Dense(10, activation='relu', input_dim=num_columns))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def define_cgan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

def generate_latent_vectors(batch_size):
    return np.random.randn(batch_size, latent_dim)

def generate_fake_samples(generator, batch_size):
    latent_vectors = generate_latent_vectors(batch_size)
    fake_samples = generator.predict(latent_vectors)
    return fake_samples

def get_random_batch(data, batch_size):
    indices = np.random.randint(0, data.shape[0], batch_size)
    real_samples = data[indices]
    return real_samples

def train_cgan(data):
    generator = define_generator()
    discriminator = define_discriminator()
    gan = define_gan(generator, discriminator)

    discriminator.compile(loss='binary_crossentropy', optimizer='adam')
    gan.compile(loss='binary_crossentropy', optimizer='adam')

    for epoch in range(num_epochs):
        real_samples = get_random_batch(data, batch_size)
        fake_samples = generate_fake_samples(generator, batch_size)
#         target = labels[:batch_size]

        labels = np.ones((batch_size, 1))
        discriminator_loss_real = discriminator.train_on_batch(real_samples, labels)

        labels = np.zeros((batch_size, 1))
        discriminator_loss_fake = discriminator.train_on_batch(fake_samples, labels)

        latent_vectors = generate_latent_vectors(batch_size)
        target = labels1[:batch_size]
        misleading_targets = np.ones((batch_size, 1))
        generator_loss = gan.train_on_batch(latent_vectors, target)

        if (epoch+1) % 100 == 0:
            print(f"Epoch: {epoch+1}, Discriminator Loss: {discriminator_loss_real+discriminator_loss_fake}, Generator Loss: {generator_loss}")
    # Generate fake samples for CSV
    fake_data = generate_fake_samples(generator, num_rows)
    fake_df = pd.DataFrame(fake_data, columns=[f'column{i}' for i in range(num_columns)])
    fake_df.to_csv('fake_data.csv', index=False)
    print("Fake data CSV generated successfully.")
            
train_cgan(data)

1/1 [==============================] - 0s 19ms/step
Epoch: 100, Discriminator Loss: 13.860143780708313, Generator Loss: -25.836721420288086
1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step
Epoch: 200, Discriminator Loss: 14.186840176582336, Generator Loss: -28.099472045898438
1/1 [==============================] - 0s 21ms/step
Epoch: 300, Discriminator Loss: 14.172695994377136, Generator Loss: -28.501094818115234
1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step
Epoch: 400, Discriminator Loss: 13.944527685642242, Generator Loss: -28.637290954589844
1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 17ms/step
Epoch: 500, Discriminator Loss: 14.093118906021118, Generator Loss: -28.697221755981445
1/1 [==============================] - 0s 21ms/step
Epoch: 600, Discriminator Loss: 14.69609159231186, Generator Loss: -28.75442886352539
1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 34ms/step
Epoch: 700, Discriminator Loss: 14.216464877128601, Generator Loss: -28.800884246826172
1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 25ms/step
Epoch: 800, Discriminator Loss: 14.438733875751495, Generator Loss: -28.807695388793945
1/1 [==============================] - 0s 31ms/step
Epoch: 900, Discriminator Loss: 14.234501719474792, Generator Loss: -28.840007781982422
1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 19ms/step
Epoch: 1000, Discriminator Loss: 13.98276001214981, Generator Loss: -28.846229553222656
14/14 [==============================] - 0s 3ms/step
Fake data CSV generated successfully.
